In [1]:
import pandas as pd
from sklearn.ensemble import GradientBoostingRegressor

# 加载数据文件
data_path = "jq_stock_ticks.csv"
data = pd.read_csv(data_path)

# 定义五档买卖价量数据作为特征
feature_columns = [col for col in data.columns if col.startswith('a') or col.startswith('b')]
features = data[feature_columns].iloc[:-100]

# 计算未来100条数据的成交均价
target_100 = data['current'].rolling(window=101).mean().shift(-100)[:-100]

# 创建并训练梯度提升回归模型
regressor = GradientBoostingRegressor(random_state=42)
regressor.fit(features, target_100)


ValueError: could not convert string to float: '2022-10-10 09:25:00'

In [37]:

from datetime import timedelta

# 加载新数据进行预测
new_data_path = "jq_stock_ticks_next.csv"
new_data = pd.read_csv(new_data_path)

new_data['timestamp'] = pd.to_datetime(new_data['time'])
new_data['date'] = new_data['timestamp'].dt.date

# 提取新数据中的五档买卖价量数据
new_features = new_data[feature_columns]

# 使用训练的模型进行预测
new_predictions = regressor.predict(new_features)

# 初始化变量
capital = 100000  # 初始资本
shares = 0  # 持有股票数量
today_purchased_shares = 0
buy_sell_log = []  # 存储买入和卖出的日志
threshold = 0.01# 买入卖出门槛
trade_count = 0  # 交易次数
prev_date = None
prev_trade_time = None  # 上一次交易时间

# 遍历预测和实际价格
for idx, (prediction, current_price, date) in enumerate(zip(new_predictions, new_data['current'], new_data['date'])):
    timestamp = new_data['timestamp'].iloc[idx]

    price_difference = prediction - current_price


    if price_difference > threshold and capital > 0:  # 买入条件
        purchase_shares = capital / current_price * (1 - 0.0003)  # 扣除交易费
        shares += purchase_shares
        capital = 0
        buy_sell_log.append((timestamp, 'buy', current_price))
        trade_count += 1
        prev_trade_time = timestamp  # 更新上一次交易时间
    elif price_difference < -threshold and shares > 0:  # 卖出条件
        capital += shares * current_price * (1 - 0.0003)  # 扣除交易费
        shares = 0
        buy_sell_log.append((timestamp, 'sell', current_price))
        trade_count += 1
        prev_trade_time = timestamp  # 更新上一次交易时间

# 最后的卖出（如果还持有股票）
if shares > 0:
    capital += shares * new_data['current'].iloc[-1] * (1 - 0.0003)
    timestamp = new_data['timestamp'].iloc[-1]
    buy_sell_log.append((timestamp, 'sell', new_data['current'].iloc[-1] ))

# 打印买入和卖出日志
for log in buy_sell_log:
    print(f"Time: {log[0]}, Action: {log[1]}, Price: {log[2]:.2f}")

initial_capital = 100000
final_capital = capital
profit_percentage = (final_capital - initial_capital) / initial_capital * 100

print(f"最终资本: {final_capital:.2f}")
print(f"收益率: {profit_percentage:.2f}%")
print(f"总交易次数: {trade_count}")

Time: 2023-04-03 09:25:00, Action: buy, Price: 1.58
Time: 2023-04-03 09:34:18, Action: sell, Price: 1.59
Time: 2023-04-03 09:34:33, Action: buy, Price: 1.58
Time: 2023-04-03 09:35:39, Action: sell, Price: 1.59
Time: 2023-04-03 09:36:06, Action: buy, Price: 1.58
Time: 2023-04-03 09:38:36, Action: sell, Price: 1.60
Time: 2023-04-03 09:41:00, Action: buy, Price: 1.59
Time: 2023-04-03 09:48:51, Action: sell, Price: 1.60
Time: 2023-04-03 09:49:09, Action: buy, Price: 1.60
Time: 2023-04-03 10:10:18, Action: sell, Price: 1.60
Time: 2023-04-03 10:10:27, Action: buy, Price: 1.59
Time: 2023-04-03 10:10:42, Action: sell, Price: 1.60
Time: 2023-04-03 10:12:00, Action: buy, Price: 1.59
Time: 2023-04-03 10:27:51, Action: sell, Price: 1.62
Time: 2023-04-03 10:33:42, Action: buy, Price: 1.60
Time: 2023-04-03 10:34:30, Action: sell, Price: 1.62
Time: 2023-04-03 10:35:57, Action: buy, Price: 1.60
Time: 2023-04-03 13:48:51, Action: sell, Price: 1.61
Time: 2023-04-03 14:00:21, Action: buy, Price: 1.60
Tim